In [1]:
import pandas as pd
import copy

In [2]:
for i in range(12,21,2):
    home_econ = r"D:\cfps\cfps"+"20%s"%i+"家庭经济.dta"
    home_rel = r"D:\cfps\cfps"+"20%s"%i+"家庭关系.dta"
    if i<17:
        adult = r"D:\cfps\cfps"+"20%s"%i+"成人库.dta"
    else:
        adult = r"D:\cfps\cfps"+"20%s"%i+"个人库.dta"
    #导入文件
    for j in (home_econ,home_rel,adult):
        a = pd.read_stata(j,convert_categoricals=False)
        #由于CFPS的value labels不唯一，因此convert_categoricals=False
        a_1 = copy.copy(a)
        #深拷贝一份，方便后面做填充
        with pd.io.stata.StataReader(j) as reader:
            lb = reader.variable_labels()
            #读取变量标签
            vl = reader.value_labels()
            #读取值标签
        for k in vl.keys():
            a[k] = a[k].map(vl[k],na_action='ignore')
            #将值标签字典映射到a，但是部分正常数据会被替换成NaN
        for k in a.columns:
            a[k].fillna(value=a_1[k], inplace=True)
            #用深拷贝的填充a的NaN        
        b = pd.DataFrame(dict(lb = lb)).T
        #变量标签导出
        c = pd.concat([b,a])
        #变量标签合并
        for l in ('家庭经济','家庭关系','个人库'):
            c.to_csv("20%s"%i+l+".csv",index=False,encoding='utf-8-sig')
        #导出为csv
                
                

